In [1]:
import numpy as np
import pandas as pd
import pingouin as pg
import matplotlib.pyplot as plt

# DATA TREATMENT

In [2]:
columns = [
    'Subject',
    'Measuring station',
    'repetition no.',
    'Neck girth',
    'Back neck point to waist',
    'Upper arm girth R',
    'Back neck point to wrist R',
    'Across back shoulder width',
    'Bust girth',
    'Waist girth',
    'Hip girth',
    'Thigh girth R',
    'Total crotch length',
    'Inside leg height',
]

In [3]:
measures_data = pd.read_csv("data/body_measurements.csv", sep=';')
measures_data = measures_data[columns]
measures_list = ['Neck girth', 'Bust girth', 'Waist girth', 'Hip girth', 'Upper arm girth R', 'Thigh girth R']
station_list=np.append(measures_data["Measuring station"].unique(), "Meu")

In [4]:
for column in columns:
    if column != 'Measuring station' and column != 'Subject':
        filtered = measures_data[column] > 2000
        measures_data[column].loc[filtered] /= 1000

In [5]:
extra_data = pd.read_csv('data/basic_informations.csv', sep=';')

In [6]:
treated_data = pd.merge(measures_data, extra_data, on='Subject')
treated_data = treated_data.drop(['Pantone', 'BMI (kg/m^2)'], axis=1)

In [7]:
treated_data[treated_data['Sex'] == 'male'].to_pickle('data/male_measures.zip')
treated_data[treated_data['Sex'] == 'female'].to_pickle('data/female_measures.zip')

In [8]:
outliers = []
subjects_list = treated_data['Subject'].unique()
for subject in subjects_list:
    for measure in measures_list:
        temp_data = treated_data[treated_data["Subject"] == subject][["Measuring station", 'Subject','repetition no.', measure]].copy()
        temp_data[measure]
        q1 = temp_data[measure].quantile(0.25)
        q3 = temp_data[measure].quantile(0.75)
        iqr = q3 - q1 #Interquartile range
        fence_low = q1 - (1.5*iqr)
        fence_high = q3 + (1.5*iqr)

        filter = (temp_data[measure] < fence_low) | (temp_data[measure] > fence_high)
        outliers.extend(temp_data[filter].index)
outliers = pd.Series(outliers)

In [9]:
outliers = []
subjects_list = treated_data['Subject'].unique()
for subject in subjects_list:
    for measure in measures_list:
        temp_data = treated_data[treated_data["Subject"] == subject][["Measuring station", 'Subject','repetition no.', measure]].copy()
        zscore = (temp_data[measure] - temp_data[measure].mean()) / (temp_data[measure].std())
        filter = abs(zscore) > 3
        outliers.extend(temp_data[filter].index)
outliers = pd.Series(outliers)
outliers = treated_data.loc[outliers]['Subject'].unique()

In [10]:
f1 = treated_data["Phase"] == 2
f2 = treated_data['repetition no.'] == 1
f3 = treated_data['Sex'] == 'female'
f4 = treated_data['Sex'] == 'male'

male_subjects = treated_data[ f1 & f2 & f4 ]['Subject'].unique()
female_subjects = treated_data[ f1 & f2 & f3 ]['Subject'].unique()

In [11]:
calculated_data_male = pd.read_csv("data/male_calculated_measures.csv").drop(['Unnamed: 0', 'age'], axis=1)
calculated_data_male*=10
calculated_data_male['repetition no.'] = np.tile(np.array([1,2]), 36)
calculated_data_male['Subject'] = male_subjects.repeat(2)
calculated_data_male['Measuring station'] = "Meu"
calculated_data_male['Phase'] = 2

calculated_data_female = pd.read_csv("data/female_calculated_measures.csv").drop(['Unnamed: 0', 'age'], axis=1)
calculated_data_female*=10
calculated_data_female['repetition no.'] = np.tile(np.array([1,2]), 36)
calculated_data_female['Subject'] = female_subjects.repeat(2)
calculated_data_female['Measuring station'] = "Meu"
calculated_data_female['Phase'] = 2

calculated_columns = list(calculated_data_female.columns[:-4])

In [12]:
treated_data = treated_data[measures_list + ['repetition no.', 'Subject', 'Measuring station', 'Phase']]
treated_data.columns = list(calculated_columns) + ['repetition no.', 'Subject', 'Measuring station', 'Phase']

In [13]:
treated_data = pd.concat([treated_data, calculated_data_male])
treated_data = pd.concat([treated_data, calculated_data_male])

In [14]:
treated_data = treated_data[~treated_data["Subject"].isin(outliers)]

In [15]:
treated_data

,neck_girth,bust_chest_girth,waist_girth,hip_girth,upper_arm_girth,thigh_girth,repetition no.,Subject,Measuring station,Phase
0,335.00000,920.00000,800.0000,1065.00000,315.00000,675.00000,1,IEEEP1_05,Expert measurer 1,1
1,345.00000,925.00000,810.0000,965.00000,310.00000,655.00000,2,IEEEP1_05,Expert measurer 1,1
2,340.00000,955.00000,847.5000,1050.00000,312.50000,670.00000,1,IEEEP1_05,Expert measurer 2,1
3,330.00000,960.00000,865.0000,1065.00000,310.00000,675.00000,2,IEEEP1_05,Expert measurer 2,1
4,335.00000,955.00000,835.0000,1045.00000,310.00000,650.00000,1,IEEEP1_05,Expert measurer 3,1
...,...,...,...,...,...,...,...,...,...,...
65,364.21585,1010.74480,889.5605,1020.42100,333.11607,583.08918,2,IEEEP2_66,Meu,2
66,342.97066,933.74480,752.4344,945.61080,330.00496,577.73900,1,IEEEP2_67,Meu,2
67,343.43678,942.60445,750.5665,947.45865,329.40380,579.43058,2,IEEEP2_67,Meu,2
70,362.16780,935.51170,900.1814,989.29596,328.05546,566.74810,1,IEEEP2_72,Meu,2


# CALC ICC

In [16]:
their_icc = pd.read_csv('data/their_icc.csv', sep=',', header=0)
their_icc = their_icc.drop(['Measuring station'], axis=1)[measures_list]
their_icc.index = ['Expert measurer 1', 'Expert measurer 2', 'Expert measurer 3', 'Expert measurer 4', 'Expert measurer 5', 'SS20', 'Vitus', 'PortalMX', 'MOVE4D', 'SS@Home', '3DLOOK', 'eM+', '3Davatarbody']

their_icc.columns = calculated_columns
their_icc.sort_index()

all_measures = []
for station in station_list:
    measures_of_station = []
    for measure in calculated_columns:
        a = treated_data[treated_data['Measuring station'] == station].reset_index()
        icc = pg.intraclass_corr(data=a, targets='Subject', raters='repetition no.', ratings=measure, nan_policy='omit')['ICC'].loc[0]
        measures_of_station.append(icc)
    all_measures.append(measures_of_station)
our_icc = pd.DataFrame(all_measures, station_list, calculated_columns).sort_index()

## OUR ICC

In [17]:
our_icc.round(3)

,neck_girth,bust_chest_girth,waist_girth,hip_girth,upper_arm_girth,thigh_girth
3DLOOK,0.998,0.989,0.992,0.992,0.990,0.985
3Davatarbody,0.983,0.987,0.996,0.994,0.985,0.992
Expert measurer 1,0.980,0.989,0.989,0.987,0.982,0.979
Expert measurer 2,0.983,0.991,0.995,0.995,0.980,0.982
Expert measurer 3,0.979,0.995,0.997,0.991,0.984,0.968
Expert measurer 4,0.984,0.990,0.991,0.991,0.979,0.983
Expert measurer 5,0.980,0.990,0.995,0.994,0.994,0.988
MOVE4D,0.996,0.994,0.997,0.999,0.996,0.998
Meu,0.993,0.995,0.999,0.998,0.996,0.963
PortalMX,0.987,0.995,0.999,0.998,0.988,0.992


## ICC DIFFERENCE

In [18]:
their_icc

,neck_girth,bust_chest_girth,waist_girth,hip_girth,upper_arm_girth,thigh_girth
Expert measurer 1,0.980,0.990,0.990,0.988,0.982,0.980
Expert measurer 2,0.983,0.990,0.995,0.995,0.980,0.982
Expert measurer 3,0.978,0.995,0.997,0.992,0.983,0.968
Expert measurer 4,0.984,0.990,0.991,0.992,0.979,0.983
Expert measurer 5,0.980,0.990,0.995,0.994,0.994,0.990
SS20,0.992,0.998,0.995,0.994,0.990,0.982
Vitus,0.994,0.997,0.997,0.998,0.976,0.997
PortalMX,0.988,0.995,0.999,0.999,0.987,0.993
MOVE4D,0.996,0.994,0.998,0.999,0.996,0.998
SS@Home,0.966,0.985,0.991,0.991,0.972,0.980


In [19]:
abs(our_icc - their_icc).round(3)

,neck_girth,bust_chest_girth,waist_girth,hip_girth,upper_arm_girth,thigh_girth
3DLOOK,0.000,0.001,0.007,0.001,0.004,0.001
3Davatarbody,0.002,0.001,0.000,0.001,0.001,0.001
Expert measurer 1,0.000,0.001,0.001,0.001,0.000,0.001
Expert measurer 2,0.000,0.001,0.000,0.000,0.000,0.000
Expert measurer 3,0.001,0.000,0.000,0.001,0.001,0.000
Expert measurer 4,0.000,0.000,0.000,0.001,0.000,0.000
Expert measurer 5,0.000,0.000,0.000,0.000,0.000,0.002
MOVE4D,0.000,0.000,0.001,0.000,0.000,0.000
Meu,NaN,NaN,NaN,NaN,NaN,NaN
PortalMX,0.001,0.000,0.000,0.001,0.001,0.001


# CALC SEM

In [20]:
their_sem = pd.read_csv('data/their_sem.csv', sep=';')[measures_list]
their_sem.index = ['Expert measurer 1', 'Expert measurer 2', 'Expert measurer 3', 'Expert measurer 4', 'Expert measurer 5', 'SS20', 'Vitus', 'PortalMX', 'MOVE4D', 'SS@Home', '3DLOOK', 'eM+', '3Davatarbody']
their_sem.columns = calculated_columns
their_sem.sort_index()


all_measures = []
for station in station_list:
    measures_of_station = []
    for measure in calculated_columns:
        a = treated_data[treated_data['Measuring station'] == station].reset_index()
        a = a[calculated_columns + ['Subject', 'repetition no.']]
        a.columns = list(calculated_columns) + ['Subject', 'repetition no.']
        icc = pg.intraclass_corr(data=a, targets='Subject', raters='repetition no.', ratings=measure, nan_policy='omit')['ICC'].loc[1]
        sem = a[measure].std() * np.sqrt(1-icc)
        measures_of_station.append(sem)
    all_measures.append(measures_of_station)
our_sem = pd.DataFrame(all_measures, station_list, calculated_columns).sort_index()

## OUR SEM

In [21]:
our_sem.round(3)

,neck_girth,bust_chest_girth,waist_girth,hip_girth,upper_arm_girth,thigh_girth
3DLOOK,1.114,10.873,10.931,7.804,3.076,6.328
3Davatarbody,4.700,11.534,8.712,7.665,4.372,5.299
Expert measurer 1,5.320,11.682,13.416,11.821,5.477,9.143
Expert measurer 2,5.335,12.673,9.949,7.662,5.971,9.477
Expert measurer 3,5.830,9.499,7.909,9.790,5.472,12.553
Expert measurer 4,5.320,13.031,13.199,9.736,6.270,8.778
Expert measurer 5,4.503,10.460,8.850,7.394,3.197,6.257
MOVE4D,2.286,7.887,6.759,2.734,2.496,2.664
Meu,2.160,4.887,2.812,2.518,1.641,17.573
PortalMX,4.058,7.247,4.209,3.757,4.569,5.445


## SEM DIFFERENCE

In [22]:
abs(our_sem - their_sem).round(3)

,neck_girth,bust_chest_girth,waist_girth,hip_girth,upper_arm_girth,thigh_girth
3DLOOK,0.086,0.273,5.469,0.204,0.824,0.672
3Davatarbody,0.100,0.234,0.112,0.065,0.072,0.001
Expert measurer 1,0.080,0.018,0.084,0.221,0.023,0.057
Expert measurer 2,0.035,0.027,0.251,0.062,0.029,0.077
Expert measurer 3,0.030,0.001,0.091,0.010,0.028,0.047
Expert measurer 4,0.020,0.059,0.099,0.036,0.030,0.022
Expert measurer 5,0.097,0.160,0.850,0.406,0.103,0.157
MOVE4D,0.014,0.013,0.259,0.034,0.004,0.064
Meu,NaN,NaN,NaN,NaN,NaN,NaN
PortalMX,0.158,0.053,0.109,0.057,0.131,0.055


# CALC PSD

In [23]:
def calculate_psd(measure, phase):
    data = treated_data[treated_data['Phase'] == phase]
    f1 = data['repetition no.'] == 1
    f2 = data['repetition no.'] == 2
    data = data[ f1 | f2 ][['Subject', 'Measuring station', 'repetition no.', measure]]
    data = data.pivot_table(values=measure, index=['Subject', 'repetition no.'], columns="Measuring station")
    data = data.dropna()
    

    column_wise = []
    for column1 in data.columns:
        row_wise = []
        for column2 in data.columns:
            if column1 == column2:
                row_wise.append(np.nan)
            else:
                d1 = data[column1]
                d2 = data[column2]
                var = (d1-d2).var()
                sem1 = our_sem[measure].loc[column1]**2
                sem2 = our_sem[measure].loc[column2]**2
                value = np.sqrt(var + (sem1/2) + (sem2/2))
                row_wise.append(value)
        column_wise.append(row_wise)
    data = pd.DataFrame(column_wise, index=data.columns, columns=data.columns)
    data.index.name = None
    return data

In [24]:
calculate_psd(measure='neck_girth', phase=2).round(1)

,3DLOOK,3Davatarbody,Expert measurer 1,Expert measurer 5,MOVE4D,Meu,PortalMX,SS@Home,eM+
3DLOOK,NaN,21.2,18.6,18.0,23.3,22.7,20.5,28.5,19.3
3Davatarbody,21.2,NaN,16.7,15.7,16.0,16.3,14.7,21.7,14.0
Expert measurer 1,18.6,16.7,NaN,8.2,11.0,12.3,12.2,16.9,14.1
Expert measurer 5,18.0,15.7,8.2,NaN,10.7,11.7,12.1,17.1,12.7
MOVE4D,23.3,16.0,11.0,10.7,NaN,5.4,10.3,14.6,13.0
Meu,22.7,16.3,12.3,11.7,5.4,NaN,10.1,15.0,13.3
PortalMX,20.5,14.7,12.2,12.1,10.3,10.1,NaN,17.6,11.4
SS@Home,28.5,21.7,16.9,17.1,14.6,15.0,17.6,NaN,21.4
eM+,19.3,14.0,14.1,12.7,13.0,13.3,11.4,21.4,NaN


In [25]:
calculate_psd(measure='bust_chest_girth', phase=2).round(1)

,3DLOOK,3Davatarbody,Expert measurer 1,Expert measurer 5,MOVE4D,Meu,PortalMX,SS@Home,eM+
3DLOOK,NaN,32.1,31.1,26.3,27.6,27.9,28.0,34.7,24.7
3Davatarbody,32.1,NaN,26.2,25.9,27.1,23.7,28.4,30.8,27.8
Expert measurer 1,31.1,26.2,NaN,21.4,21.2,22.0,23.4,29.7,26.2
Expert measurer 5,26.3,25.9,21.4,NaN,17.6,18.8,16.8,27.8,24.9
MOVE4D,27.6,27.1,21.2,17.6,NaN,19.5,16.7,27.6,25.7
Meu,27.9,23.7,22.0,18.8,19.5,NaN,19.9,25.8,24.6
PortalMX,28.0,28.4,23.4,16.8,16.7,19.9,NaN,28.3,28.5
SS@Home,34.7,30.8,29.7,27.8,27.6,25.8,28.3,NaN,32.4
eM+,24.7,27.8,26.2,24.9,25.7,24.6,28.5,32.4,NaN


In [26]:
calculate_psd(measure='waist_girth', phase=2).round(1)

,3DLOOK,3Davatarbody,Expert measurer 1,Expert measurer 5,MOVE4D,Meu,PortalMX,SS@Home,eM+
3DLOOK,NaN,40.8,26.2,27.7,26.1,32.6,22.3,43.5,28.4
3Davatarbody,40.8,NaN,34.9,31.9,28.4,25.7,29.8,47.1,34.6
Expert measurer 1,26.2,34.9,NaN,17.4,20.1,26.6,19.1,39.2,25.3
Expert measurer 5,27.7,31.9,17.4,NaN,19.3,24.2,19.1,39.4,24.6
MOVE4D,26.1,28.4,20.1,19.3,NaN,18.4,14.1,41.7,22.9
Meu,32.6,25.7,26.6,24.2,18.4,NaN,23.7,33.3,30.8
PortalMX,22.3,29.8,19.1,19.1,14.1,23.7,NaN,41.5,20.5
SS@Home,43.5,47.1,39.2,39.4,41.7,33.3,41.5,NaN,45.8
eM+,28.4,34.6,25.3,24.6,22.9,30.8,20.5,45.8,NaN


In [27]:
calculate_psd(measure='hip_girth', phase=2).round(1)

,3DLOOK,3Davatarbody,Expert measurer 1,Expert measurer 5,MOVE4D,Meu,PortalMX,SS@Home,eM+
3DLOOK,NaN,26.9,24.7,22.5,20.3,20.0,22.5,25.1,23.0
3Davatarbody,26.9,NaN,23.8,23.6,20.0,21.5,18.5,22.4,22.5
Expert measurer 1,24.7,23.8,NaN,16.7,13.1,13.5,14.5,19.1,25.0
Expert measurer 5,22.5,23.6,16.7,NaN,11.7,13.3,14.6,16.3,20.9
MOVE4D,20.3,20.0,13.1,11.7,NaN,8.4,9.7,13.5,20.1
Meu,20.0,21.5,13.5,13.3,8.4,NaN,10.2,14.8,21.9
PortalMX,22.5,18.5,14.5,14.6,9.7,10.2,NaN,16.1,22.3
SS@Home,25.1,22.4,19.1,16.3,13.5,14.8,16.1,NaN,23.0
eM+,23.0,22.5,25.0,20.9,20.1,21.9,22.3,23.0,NaN


In [28]:
calculate_psd(measure='upper_arm_girth', phase=2).round(1)

,3DLOOK,3Davatarbody,Expert measurer 1,Expert measurer 5,MOVE4D,Meu,PortalMX,SS@Home,eM+
3DLOOK,NaN,17.1,20.3,16.5,16.9,14.8,19.4,18.6,32.8
3Davatarbody,17.1,NaN,18.1,14.8,14.4,14.1,16.4,15.7,33.4
Expert measurer 1,20.3,18.1,NaN,12.0,9.1,9.9,11.2,13.9,31.9
Expert measurer 5,16.5,14.8,12.0,NaN,7.2,8.6,9.1,15.3,33.8
MOVE4D,16.9,14.4,9.1,7.2,NaN,5.3,8.0,12.3,32.0
Meu,14.8,14.1,9.9,8.6,5.3,NaN,9.9,11.4,30.0
PortalMX,19.4,16.4,11.2,9.1,8.0,9.9,NaN,15.6,36.2
SS@Home,18.6,15.7,13.9,15.3,12.3,11.4,15.6,NaN,28.1
eM+,32.8,33.4,31.9,33.8,32.0,30.0,36.2,28.1,NaN


In [29]:
calculate_psd(measure='thigh_girth', phase=2).round(1)

,3DLOOK,3Davatarbody,Expert measurer 1,Expert measurer 5,MOVE4D,Meu,PortalMX,SS@Home,eM+
3DLOOK,NaN,24.2,25.5,25.1,25.8,69.4,24.0,31.8,24.5
3Davatarbody,24.2,NaN,17.9,18.2,14.6,73.6,18.6,22.5,19.6
Expert measurer 1,25.5,17.9,NaN,11.8,10.4,72.2,12.8,20.3,16.6
Expert measurer 5,25.1,18.2,11.8,NaN,11.2,70.8,12.0,20.6,16.5
MOVE4D,25.8,14.6,10.4,11.2,NaN,70.6,10.8,17.6,14.1
Meu,69.4,73.6,72.2,70.8,70.6,NaN,69.5,72.5,69.2
PortalMX,24.0,18.6,12.8,12.0,10.8,69.5,NaN,20.9,15.1
SS@Home,31.8,22.5,20.3,20.6,17.6,72.5,20.9,NaN,21.4
eM+,24.5,19.6,16.6,16.5,14.1,69.2,15.1,21.4,NaN
